In [ ]:
df=read.csv("C:/Users/student/Downloads/R/adult.csv", header=FALSE)
names(df)=c('age','workclass','fnlwt','education','education_num','marital_status','occupation','relationship','race','sex','capital_gain','capital_loss','hours_per_week','native_country','target')
str(df)
df[df==" ?"]=NA

misshandled=function(data){
  for(i in 1:ncol(data)){
    if (!is.numeric(data[,i])){
      levels=unique(data[,i])
      data[,i][is.na(data[,i])]=levels[which.max(tabulate(match(data[,i],levels)))]
    }
  }
  return(data)
}
newdf=misshandled(df)
sum(is.na(newdf))
levels(newdf$education)=c("Before HS","Before HS", "Before HS", "Before HS","Before HS","Before HS","Before HS","Before HS","Some College", "Some College", "Bachelors", "Higher College", "Some HS", "Higher College","Before HS","Higher College","Some College")
                  

levels(newdf$native_country)=c("Asia","Asia", "North America", "Asia", "South America", "Central America", "Central America", "South America", "Central America", "Europe", "Europe", "Europe", "Europe", "Central America", "Central America", "Europe", "Central America", "Asia", "Europe", "Asia", "Asia", "Europe", "Europe", "Central America", "Asia", "Asia", "North America", "Central America", "North America", "South America", "Asia", "Europe", "Europe","Central America", "Europe", "Asia", "Asia", "Asia", "Central America", "North America", "Asia", "Europe")

levels(newdf$occupation)=c("Craft","Office", "Army","Craft", "Office","Craft", "Craft", "Craft", "Other", "Craft", "Craft", "Army", "Sales", "Craft", "Craft")

titanic=read.csv("C:/Users/student/Downloads/R/titanic.csv")
titanic$Survived = factor(titanic$Survived)
titanic$Pclass = factor(titanic$Pclass)

misshandled=function(data){
  for(i in 1:ncol(data)){
    if (is.numeric(data[,i])){
      data[,i][is.na(data[,i])]=mean(data[,i], na.rm=TRUE)
    } else{
      levels=unique(data[,i])
      data[,i][is.na(data[,i])]=levels[which.max(tabulate(match(data[,i],levels)))]
    }
  }
  return(data)
}
titanic1=misshandled(titanic)
str(titanic1)


#1.	Preprocess the data so that it is ready to train predictive models.

library(caret)
library(readr)
preProcess_missingdata_model <- preProcess(newdf, method='medianImpute')
preProcess_missingdata_model

#2.	Train a random forest with 7-fold cross validation.  Report the accuracy of the forest. 

```{r , echo=TRUE}
myGrid = expand.grid(mtry = 3, splitrule = c("gini"),
                     min.node.size = c(1:3))
model <- train(
  target~.,
  tuneLength = 1,
  data = newdf, method = "ranger",
  trControl = trainControl(method = "cv", number = 7, verboseIter = TRUE),tuneGrid=myGrid
)
pred=predict(model,newdf)
levels(newdf$target) = c("0", "1")
levels(pred) = c("0", "1")
cm=confusionMatrix(pred, newdf$target, positive="1")
cm

#2b. titanic data

myGrid = expand.grid(mtry = 3, splitrule = c("gini"),
                     min.node.size = c(1:3))
model1 <- train(
  Survived~.,
  tuneLength = 1,
  data = titanic1, method = "ranger",
  trControl = trainControl(method = "cv", number = 7, verboseIter = TRUE),tuneGrid=myGrid
)
pred1=predict(model1,titanic1)
levels(titanic1$Survived) = c("0", "1")
levels(pred1) = c("0", "1")
cm1=confusionMatrix(pred1, titanic1$Survived, positive="1")
cm1

#3.	Train a decision tree with 10-fold cross validation.  Report the accuracy. 



model2 <- train(
  target~.,
  tuneLength = 1,
  data = newdf, method = "rpart",
  trControl = trainControl(method = "cv", number = 10, verboseIter = TRUE)
)
pred2=predict(model2,newdf)
levels(newdf$target) = c("0", "1")
levels(pred2) = c("0", "1")
cm2=confusionMatrix(pred2, newdf$target, positive="1")
cm2

#3b. titanic data

model3 <- train(
  Survived~.,
  tuneLength = 1,
  data = titanic1, method = "rpart",
  trControl = trainControl(method = "cv", number = 10, verboseIter = TRUE)
)
pred3=predict(model3,titanic1)
levels(titanic1$Survived) = c("0", "1")
levels(pred) = c("0", "1")
cm3=confusionMatrix(pred3, titanic1$Survived, positive="1")
cm3

model
model1
plot(model)
plot(model1)

6.	Tune the three hyperparameters of the random forest with mtry running all its possible values, splitrule running all its possible values (gini and extratrees) and min.node.size running from 1 to 20

myGrid = expand.grid(mtry = 3, splitrule = c("gini", "extratrees"),
                     min.node.size = c(1:4))
model4 <- train(
  target~.,
  tuneLength = 1,
  data = newdf, method = "ranger",
  trControl = trainControl(method = "cv", number = 4, verboseIter = TRUE),tuneGrid=myGrid)
pred4=predict(model4,newdf)
levels(newdf$target) = c("0", "1")
levels(pred4) = c("0", "1")
cm4=confusionMatrix(pred4, newdf$target, positive="1")
cm4

6b. titanic data
myGrid = expand.grid(mtry = 4, splitrule = c("gini", "extratrees"),
                     min.node.size = c(1:4))
model5 <- train(
  Survived~.,
  tuneLength = 1,
  data = titanic1, method = "ranger",
  trControl = trainControl(method = "cv", number = 7, verboseIter = TRUE),tuneGrid=myGrid)
pred5=predict(model5,titanic1)
levels(titanic1$Survived) = c("0", "1")
levels(pred5) = c("0", "1")
cm5=confusionMatrix(pred5, titanic1$Survived, positive="1")
cm5

plot(model4)
plot(model5)

#8.	Compare the performances of the default glmnet model and a tuned glmnet model. Do you achieve better performance in the tuned glmnet model? 

library(glmnet)
model6 <- train(
  target~.,
  tuneLength=1,
  data = newdf, method = "glmnet",
  trControl = trainControl(method = "cv", number = 7, verboseIter = TRUE))
pred6=predict(model6,newdf)
levels(newdf$target) = c("0", "1")
levels(pred6) = c("0", "1")
cm6=confusionMatrix(pred6, newdf$target, positive="1")
cm6

myGrid = expand.grid(alpha = 0:1,lambda = seq(0.0001,1,length = 10))

model7 <- train(
  target~.,
  tuneLength = 1,
  data = newdf, method = "glmnet",
  trControl = trainControl(method = "cv", number = 4, verboseIter = TRUE),tuneGrid=myGrid)
pred7=predict(model7,newdf)
levels(newdf$target) = c("0", "1")
levels(pred7) = c("0", "1")
cm7=confusionMatrix(pred7, newdf$target, positive="1")
cm7

myGrid = expand.grid(alpha = 0:1,lambda = seq(0.0001,1,length = 10))

model8 <- train(
  Survived~.,
  tuneLength = 1,
  data = titanic1, method = "glmnet",
  trControl = trainControl(method = "cv", number = 7, verboseIter = TRUE),tuneGrid=myGrid)
pred8=predict(model8,titanic1)
levels(titanic1$Survived) = c("0", "1")
levels(pred8) = c("0", "1")
cm8=confusionMatrix(pred8, titanic1$Survived, positive="1")
cm8

model9 <- train(
  Survived~.,
  tuneLength = 1,
  data = titanic1, method = "glmnet",
  trControl = trainControl(method = "cv", number = 7, verboseIter = TRUE))
pred9=predict(model9,titanic1)
levels(titanic1$Survived) = c("0", "1")
levels(pred9) = c("0", "1")
cm9=confusionMatrix(pred9, titanic1$Survived, positive="1")
cm9